<a href="https://colab.research.google.com/github/LucasSerranog/IA_udea/blob/main/04_modelo_con_preprocesado_de_otra_forma_y_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo con preprocesado alternativo y Random Forest

#integrante
Lucas Serrano CC.1000944832


En este notebook:
- Aplicaremos un preprocesado distinto al utilizado en el notebook 03.
- Entrenaremos un modelo Random Forest para predecir `RENDIMIENTO_GLOBAL`.
- Evaluaremos su rendimiento y lo compararemos con el modelo SVM.

Cargamos las librerias necesarias

In [9]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

subimos los datos preprocesados

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ruta = "/content/drive/MyDrive/datos_preprocesados.csv"
d_p = pd.read_csv(ruta)


Se cargan los datos preprocesados

In [4]:
d_p

,FAMI_EDUCACIONMADRE__Educación profesional completa,FAMI_EDUCACIONMADRE__Educación profesional incompleta,FAMI_EDUCACIONMADRE__Ninguno,FAMI_EDUCACIONMADRE__Postgrado,FAMI_EDUCACIONMADRE__Primaria completa,FAMI_EDUCACIONMADRE__Primaria incompleta,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE__Técnica o tecnológica completa,FAMI_EDUCACIONMADRE__Técnica o tecnológica incompleta,...,ESTU_VALORMATRICULAUNIVERSIDAD__No pagó matrícula,FAMI_TIENEINTERNET__No,FAMI_TIENEINTERNET__Si,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,ENFERMERIA,BOGOTÁ,medio-alto,0.322,0.208,0.310,0.267
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,DERECHO,ATLANTICO,bajo,0.311,0.215,0.292,0.264
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,MERCADEO Y PUBLICIDAD,BOGOTÁ,bajo,0.297,0.214,0.305,0.264
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,ADMINISTRACION DE EMPRESAS,SANTANDER,alto,0.485,0.172,0.252,0.190
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,PSICOLOGIA,ANTIOQUIA,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,BIOLOGIA,LA GUAJIRA,medio-alto,0.237,0.271,0.271,0.311
692496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,PSICOLOGIA,NORTE SANTANDER,bajo,0.314,0.240,0.278,0.260
692497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,medio-bajo,0.286,0.240,0.314,0.287
692498,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,PSICOLOGIA,TOLIMA,bajo,0.132,0.426,0.261,0.328


Eliminamos columnas irrelevantes como lo son el programa academico debido a la variabilidad de esos datos, al igual que el departamento

In [5]:
# Eliminar columnas irrelevantes
columnas_a_eliminar = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']
d_p = d_p.drop(columns=columnas_a_eliminar)

## Preparación de datos
En esta sección:
- Separamos la variable objetivo `RENDIMIENTO_GLOBAL` de las características.
- Dividimos los datos en conjuntos de entrenamiento y evaluación.


In [6]:
from sklearn.model_selection import train_test_split

X = d_p.drop(columns='RENDIMIENTO_GLOBAL')
y = d_p['RENDIMIENTO_GLOBAL']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,  stratify=y)

print(f"Entrenamiento: {X_train.shape}, Prueba: {X_test.shape}")



Entrenamiento: (554000, 54), Prueba: (138500, 54)


## Entrenamiento del modelo Random Forest
Utilizamos Random Forest para capturar relaciones no lineales en los datos.
Ajustamos un modelo estándar para evaluar su rendimiento.


In [14]:
# Submuestreo para búsqueda rápida
X_sub, _, y_sub, _ = train_test_split(X_train, y_train, train_size=10000, random_state=42, stratify=y_train)

param_dist = {
    'n_estimators': [100, 300],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

modelo_rf = RandomForestClassifier(random_state=42)

In [15]:
random_search = RandomizedSearchCV(modelo_rf,
                                   param_distributions=param_dist,
                                   n_iter=5,
                                   cv=3,
                                   scoring='accuracy',
                                   n_jobs=-1,
                                   verbose=2)

# Ejecutamos la búsqueda en menos datos
random_search.fit(X_sub, y_sub)

# Mejor modelo encontrado
mejor_modelo_rf = random_search.best_estimator_
print("\n✅ Mejor modelo encontrado:", random_search.best_params_)


Fitting 3 folds for each of 5 candidates, totalling 15 fits

✅ Mejor modelo encontrado: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10}


## Evaluación del modelo
Realizamos predicciones en el conjunto de evaluación para evaluar la precisión alcanzada por el modelo Random Forest.


In [17]:
# Realizamos predicciones en el conjunto de test
y_pred = mejor_modelo_rf.predict(X_test)

# Evaluamos el modelo
precision = accuracy_score(y_test, y_pred)
reporte = classification_report(y_test, y_pred, digits=2)
matriz_confusion = confusion_matrix(y_test, y_pred)

# Mostramos los resultados
print(f"Precisión en el conjunto de test: {precision:.4f}\n")
print("Reporte de clasificación:\n", reporte)
print("Matriz de confusión:\n", matriz_confusion)


Precisión en el conjunto de test: 0.3777

Reporte de clasificación:
               precision    recall  f1-score   support

        alto       0.48      0.55      0.51     35124
        bajo       0.39      0.47      0.43     34597
  medio-alto       0.29      0.20      0.24     34324
  medio-bajo       0.30      0.29      0.29     34455

    accuracy                           0.38    138500
   macro avg       0.37      0.38      0.37    138500
weighted avg       0.37      0.38      0.37    138500

Matriz de confusión:
 [[19300  4424  6275  5125]
 [ 4328 16142  4808  9319]
 [10117  8510  6908  8789]
 [ 6363 12183  5945  9964]]


## Conclusiones
- El modelo Random Forest alcanzó una precisión de 0.3777.
- Se observa que 0.3907 en comparación con el modelo SVM del notebook 03.
- Se recomienda SVM para futuras iteraciones.
